In [1]:
from load_split import load_and_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras import models
from keras import layers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix,classification_report

2024-09-26 09:40:34.196019: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = load_and_split('fer_no_nf.csv')
class_weights = np.load('class_weights.npy')
class_labels = ('neutral','happiness','surprise','sadness','anger','disgust','fear','contempt','unknown')

In [9]:
epochs = 10
lr_start = 0.001
lr = 0.0003
batch_size = 32
batches = x_train.shape[0]//batch_size + 1

In [ ]:
# best parameters
# 3 conv layers, 128 filters then 64 then 32, 128 dense, 0.4 dropout, 0.0003 learning rate, 20 epochs

In [11]:
inputs = layers.Input(shape=(48, 48, 1))

x = layers.Rescaling(1./255)(inputs)

x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.BatchNormalization()(x)

x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.BatchNormalization()(x)

x = layers.Conv2D(32, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.BatchNormalization()(x)

x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(9, activation='softmax')(x)

model = models.Model(inputs=inputs, outputs=outputs)

#cosine_annealed_lr = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate = lr_start, decay_steps = epochs*batches)
optimizer = tf.keras.optimizers.Adam(learning_rate = lr)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_3 (Rescaling)         │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 46, 46, 128)    │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 23, 23, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 21, 21, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 10, 10, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 32)       │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 4, 4, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 4, 4, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 161,257 (629.91 KB)

 Trainable params: 160,809 (628.16 KB)

 Non-trainable params: 448 (1.75 KB)

In [12]:
history = model.fit(x_train, y_train,                  
                    epochs=epochs,
                    batch_size = batch_size,
                    validation_data=(x_val , y_val))
                    #class_weight={index:weight for index,weight in enumerate(class_weights)})
model.save('optimized.keras')

Epoch 1/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 300s 330ms/step - accuracy: 0.3868 - loss: 1.9447 - val_accuracy: 0.5693 - val_loss: 1.4753
Epoch 2/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 286s 321ms/step - accuracy: 0.5845 - loss: 1.4818 - val_accuracy: 0.6262 - val_loss: 1.3337
Epoch 3/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 267s 300ms/step - accuracy: 0.6351 - loss: 1.3705 - val_accuracy: 0.6215 - val_loss: 1.3930
Epoch 4/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 272s 305ms/step - accuracy: 0.6684 - loss: 1.2962 - val_accuracy: 0.6666 - val_loss: 1.2604
Epoch 5/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 296s 332ms/step - accuracy: 0.6960 - loss: 1.2481 - val_accuracy: 0.6862 - val_loss: 1.1999
Epoch 6/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 290s 325ms/step - accuracy: 0.7159 - loss: 1.2036 - val_accuracy: 0.6980 - val_loss: 1.1657
Epoch 7/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 290s 325ms/step - accuracy: 0.7235 - loss: 1.1768 - val_accuracy: 0.6918 - val_loss: 1.2134
Epoch 8/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 289s 324ms/step - accuracy: 0.7354 -

In [13]:
history = model.fit(x_train, y_train,                  
                    epochs=epochs,
                    batch_size = batch_size,
                    validation_data=(x_val , y_val))
                    #class_weight={index:weight for index,weight in enumerate(class_weights)})
model.save('optimized.keras')

Epoch 1/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 291s 326ms/step - accuracy: 0.7684 - loss: 1.0962 - val_accuracy: 0.7117 - val_loss: 1.1499
Epoch 2/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 296s 333ms/step - accuracy: 0.7729 - loss: 1.0767 - val_accuracy: 0.7277 - val_loss: 1.1158
Epoch 3/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 285s 320ms/step - accuracy: 0.7849 - loss: 1.0615 - val_accuracy: 0.7324 - val_loss: 1.1179
Epoch 4/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 275s 308ms/step - accuracy: 0.7868 - loss: 1.0590 - val_accuracy: 0.7260 - val_loss: 1.1452
Epoch 5/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 291s 327ms/step - accuracy: 0.7951 - loss: 1.0339 - val_accuracy: 0.7240 - val_loss: 1.1064
Epoch 6/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 319s 323ms/step - accuracy: 0.8010 - loss: 1.0302 - val_accuracy: 0.7296 - val_loss: 1.1196
Epoch 7/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 318s 357ms/step - accuracy: 0.8067 - loss: 1.0231 - val_accuracy: 0.7473 - val_loss: 1.0914
Epoch 8/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 291s 326ms/step - accuracy: 0.8149 -